In [18]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn

In [19]:
df = pd.read_csv('Resources/tweet_data.csv')

In [20]:
X = df['tidy_tweet'].astype(str)
ylabels = (df['target']/4).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

In [21]:
X_val = X_train[:10000]
y_val = y_train[:10000]

In [22]:
run_name = "Twitter_Sentiment"
with mlflow.start_run(run_name=run_name) as run:
    model = "https://tfhub.dev/google/universal-sentence-encoder/4"
    hub_layer = hub.KerasLayer(model, output_shape=[512], input_shape=[], 
                           dtype=tf.string, trainable=True)
    
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=20000,
                    validation_data=(X_val, y_val),
                    verbose=1)

    #Save so that we can easily load this in MLflow later if we need to
    mlflow.sklearn.save_model(model, "saved_model", 
                              serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    mlflow.log_artifact("saved_model")

Epoch 1/10
42/42 [==============================] - 144s 3s/step - loss: 0.5114 - accuracy: 0.7628 - val_loss: 0.4049 - val_accuracy: 0.7758
Epoch 2/10
42/42 [==============================] - 157s 4s/step - loss: 0.3627 - accuracy: 0.8411 - val_loss: 0.3210 - val_accuracy: 0.8656
Epoch 3/10
42/42 [==============================] - 166s 4s/step - loss: 0.3049 - accuracy: 0.8729 - val_loss: 0.2682 - val_accuracy: 0.8891
Epoch 4/10
42/42 [==============================] - 179s 4s/step - loss: 0.2548 - accuracy: 0.8993 - val_loss: 0.2062 - val_accuracy: 0.9250
Epoch 5/10
42/42 [==============================] - 177s 4s/step - loss: 0.1976 - accuracy: 0.9282 - val_loss: 0.1465 - val_accuracy: 0.9556
Epoch 6/10
42/42 [==============================] - 171s 4s/step - loss: 0.1471 - accuracy: 0.9509 - val_loss: 0.1038 - val_accuracy: 0.9675
Epoch 7/10
42/42 [==============================] - 167s 4s/step - loss: 0.1010 - accuracy: 0.9698 - val_loss: 0.0749 - val_accuracy: 0.9817
Epoch 8/10
42

MlflowException: Path 'saved_model' already exists

In [ ]:
!cat saved_model/conda.yaml